<a href="https://colab.research.google.com/github/kimjaehwankimjaehwan/Dacon/blob/main/%EC%95%94%ED%99%98%EC%9E%90_%EC%9C%A0%EC%A0%84%EC%B2%B4_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EA%B8%B0%EB%B0%98_%EC%95%94%EC%A2%85_%EB%B6%84%EB%A5%98_AI_%EB%AA%A8%EB%8D%B8_%EA%B0%9C%EB%B0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##베이스라인 코드
암환자 유전체 데이터 기반 암종 분류 AI 모델 개발
'2024 생명연구자원 AI활용 경진대회'는 바이오 데이터를 기반으로 한 AI 기술의 문제 해결 능력을 탐구하는 것을 목표로 합니다.
이 대회는 바이오 분야에서 AI 활용의 저변을 확대하고, 복잡한 바이오 데이터를 효율적으로 분석 및 해석할 수 있는 AI 알고리즘 개발에 초점을 맞추고 있습니다.

본 대회의 구체적인 과제는 암환자 유전체 데이터의 변이 정보를 활용하여 암종을 분류하는 AI 모델을 개발하는 것입니다.
참가자들은 제공된 학습 데이터셋(암환자 유전체 변이 정보)을 사용하여 특정 변이 정보를 바탕으로 암종을 정확하게 분류할 수 있는 AI 알고리즘을 개발해야 합니다.

이 대회의 궁극적인 목적은 바이오 데이터의 활용도를 높이고, 바이오 분야에서 AI 기술의 적용 가능성을 극대화하며, 인공지능 기술이 실제 바이오 의료 문제 해결에 어떻게 기여할 수 있는지 탐구하는 것입니다.

In [1]:
!unzip '/content/open (1).zip' -d '/content/'

Archive:  /content/open (1).zip
  inflating: /content/sample_submission.csv  
  inflating: /content/test.csv       
  inflating: /content/train.csv      


In [4]:
# Import Library
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import xgboost as xgb
# Load Data
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")


In [6]:
train.head() , test.head()

(           ID SUBCLASS    A2M AAAS AADAT AARS1 ABAT ABCA1 ABCA2 ABCA3  ...  \
 0  TRAIN_0000    KIPAN     WT   WT    WT    WT   WT    WT    WT    WT  ...   
 1  TRAIN_0001     SARC     WT   WT    WT    WT   WT    WT    WT    WT  ...   
 2  TRAIN_0002     SKCM  R895R   WT    WT    WT   WT    WT    WT    WT  ...   
 3  TRAIN_0003     KIRC     WT   WT    WT    WT   WT    WT    WT    WT  ...   
 4  TRAIN_0004   GBMLGG     WT   WT    WT    WT   WT    WT    WT    WT  ...   
 
   ZNF292 ZNF365 ZNF639 ZNF707 ZNFX1 ZNRF4 ZPBP ZW10 ZWINT ZYX  
 0     WT     WT     WT     WT    WT    WT   WT   WT    WT  WT  
 1     WT     WT     WT     WT    WT    WT   WT   WT    WT  WT  
 2     WT     WT     WT     WT    WT    WT   WT   WT    WT  WT  
 3     WT     WT     WT     WT    WT    WT   WT   WT    WT  WT  
 4     WT     WT     WT     WT    WT    WT   WT   WT    WT  WT  
 
 [5 rows x 4386 columns],
           ID A2M AAAS AADAT AARS1 ABAT  ABCA1 ABCA2 ABCA3 ABCA4  ... ZNF292  \
 0  TEST_0000  WT   WT    

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6201 entries, 0 to 6200
Columns: 4386 entries, ID to ZYX
dtypes: object(4386)
memory usage: 207.5+ MB


In [9]:
train.columns

Index(['ID', 'SUBCLASS', 'A2M', 'AAAS', 'AADAT', 'AARS1', 'ABAT', 'ABCA1',
       'ABCA2', 'ABCA3',
       ...
       'ZNF292', 'ZNF365', 'ZNF639', 'ZNF707', 'ZNFX1', 'ZNRF4', 'ZPBP',
       'ZW10', 'ZWINT', 'ZYX'],
      dtype='object', length=4386)

In [ ]:
# Data Preprocessing
# SUBCLASS 가 범주형이기 때문에 LabelEncoder 사용
le_subclass = LabelEncoder()
train['SUBCLASS'] = le_subclass.fit_transform(train['SUBCLASS'])



In [ ]:
# 변환된 레이블 확인
for i, label in enumerate(le_subclass.classes_):
    print(f"원래 레이블: {label}, 변환된 숫자: {i}")
## x 의 경우도 범주형으로 구성되어 있어, 알맞은 인코딩 필요
X = train.drop(columns=['SUBCLASS', 'ID'])
y_subclass = train['SUBCLASS']

categorical_columns = X.select_dtypes(include=['object', 'category']).columns
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_encoded = X.copy()
X_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])


In [ ]:
# Model Define and Train
model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss'
)

model.fit(X_encoded, y_subclass)


In [ ]:
# Inference
test_X = test.drop(columns=['ID'])
X_encoded = test_X.copy()
X_encoded[categorical_columns] = ordinal_encoder.transform(test_X[categorical_columns])
predictions = model.predict(X_encoded)
original_labels = le_subclass.inverse_transform(predictions)


In [ ]:
# Submission
submission = pd.read_csv("./sample_submission.csv")
submission["SUBCLASS"] = original_labels
submission.to_csv('./baseline_submission.csv', encoding='UTF-8-sig', index=False)